In [1]:
# Qua creo le finestre che poi verranno date in pasto alla NN
# per come è nell'articolo, io divido il segnale iniziale in finestre di 20 samples (10 ms)
# Otterrò righe m1(0), ... , mn(12) , ... , m1(k), ... mn(k), BASOGRAFICO
# m = muscolo, n = numero di muscoli, k = tempo totale di finestra 
# Nel mio caso con 10 muscoli, 2 goniometri e 1 basografico ogni riga è di 241 elementi: 240 features + 1 flag

#Le transizioni vengono scartate in base alla threshold.

import csv
import statistics
from statistics import mode 
import pandas as pd
import os

#in_path = "../All_csv_subjects/Subjects_csv/normalized/"
#out_path = "../All_csv_subjects/Subjects_Windows_20_csv/normalized/no_transitions/"
#out_path_discarded = out_path + "discarded/"
#prefix = 'Subject'
#suffix = '_norm'
#out_suffix = '_norm_windows_'

sliding = True
sliding_pace = 20
predict_central_sample = False

discard_transitions = True # Devo eliminare le finestre che contengono le transizioni
discarded_windows = 0
th = 0.8

window_size = 20
features_number = 12 # Tutto tranne il basografico
row_count = 486600

start = 11
end = 11


in_path = "./min-max/"
out_path = "./min-max/windows_" + str(window_size) +"/tr-" + str(discard_transitions) + "_sliding_" + str(sliding_pace) + "_c-" + str(predict_central_sample) + "/"
out_path_discarded = out_path + "discarded/"
prefix = 's'
suffix = '_norm'
out_suffix = '_norm_windows_'


#create folders if not existing...
if not os.path.exists(out_path):
    os.makedirs(out_path)
if not os.path.exists(out_path_discarded):
    os.makedirs(out_path_discarded)

for subj in range(start, end + 1):
    
    #count rows in file
    with open(in_path + prefix + str(subj) + suffix + '.csv') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        row_count = sum(1 for row in csv_reader)
    
    csv_in = []
    with open(in_path + prefix + str(subj) + suffix + '.csv') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        for line in csv_reader: 
            csv_in.append(line)
        
    
    with open(out_path + prefix + str(subj) + out_suffix + str(window_size) + '.csv', mode='w') as csv_out_file:
        with open(out_path_discarded + prefix + str(subj) + out_suffix + str(window_size) + '.txt', mode='w') as discarded_file:
            csv_writer = csv.writer(csv_out_file, delimiter=',')

            print("Subject:" + str(subj) + " rows: " + str(row_count))

            discarded_windows = 0

            if sliding:
                pace = sliding_pace
                limit = row_count-window_size
            else:
                pace = window_size
                limit = row_count

            for k in range(0,limit,pace):
                #print(k)
                window_rows = []
                for i in range(k, k + window_size):
                    window_rows.append(csv_in[i])
                window_rows = pd.DataFrame(window_rows).astype(float)
                # Qui abbiamo più features, creo 4 'bsx' e 4 'm' da appendere
                bsx1 = window_rows[features_number]    # Baso 2_Left
               # bsx2 = window_rows[features_number+1]  # Baso 4 Left
              #  bsx3 = window_rows[features_number+2]  # Baso 2 Right
              #  bsx4 = window_rows[features_number+3]  # Baso 4 Right
                if predict_central_sample:
                    m = bsx[int(len(bsx)/2)]
                else:
                    m1 = bsx1.mode()[0]
                   # m2 = bsx2.mode()[0]
                   # m3 = bsx3.mode()[0]
                   # m4 = bsx4.mode()[0]
               # print(m2)
                occ1 = len(window_rows[window_rows[features_number] == float(m1)])
               # occ2 = len(window_rows[window_rows[features_number + 1] == float(m2)])
               # occ3 = len(window_rows[window_rows[features_number] == float(m3)])
               # occ4 = len(window_rows[window_rows[features_number] == float(m4)])
                if not discard_transitions or occ1/window_size >= th:       # Qua dobbiamo capire come scartarle le finestre (cioè che threshold usare!)
                    single_row_window = window_rows.iloc[:, :-4].values.reshape(1,-4).flatten().tolist()
                    single_row_window.append(float(m1))
                   # single_row_window.append(float(m2))
                   # single_row_window.append(float(m3))
                   # single_row_window.append(float(m4))
                    csv_writer.writerow(single_row_window)
                else:
                    discarded_windows += 1
                    # print(discarded_windows) # Solo per vedere se cicla troppo o troppo poco
                    discarded_file.write("Window starting at row " + str(k) + "\n")
                    discarded_file.write(str(window_rows) + "\n\n\n")
            print("Discarded windows: " + str(discarded_windows))
                    

Subject:11 rows: 276581
Discarded windows: 147


In [ ]:
print(m2)
print(occ2)
a = window_rows[window_rows[features_number-1]]
print(a)